In [5]:
import pandas as pd
import numpy as np
import sklearn
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split

## Data import

In [6]:
# Data Read
df_train = pd.read_csv('../../data/raw/teco-psda-exercisesheet1-employment-2024/employment_train.csv')
df_test = pd.read_csv('../../data/raw/teco-psda-exercisesheet1-employment-2024/employment_test.csv')

# Compare Columns to Desc
print(df_test.columns)
print(df_train.columns)

Index(['Unnamed: 0', 'age', 'race', 'earnwke', 'married', 'union', 'ne_states',
       'so_states', 'ce_states', 'we_states', 'government', 'private', 'self',
       'educ_lths', 'educ_hs', 'educ_somecol', 'educ_aa', 'educ_bac',
       'educ_adv', 'female'],
      dtype='object')
Index(['Unnamed: 0', 'age', 'race', 'earnwke', 'employed', 'unemployed',
       'married', 'union', 'ne_states', 'so_states', 'ce_states', 'we_states',
       'government', 'private', 'self', 'educ_lths', 'educ_hs', 'educ_somecol',
       'educ_aa', 'educ_bac', 'educ_adv', 'female'],
      dtype='object')


In [54]:
# First Look at the data

train = df_train.values
train_ids = train[:, 0]
train_employed = train[:, 4]
train_unemployed = train[:, 5]
train_data = train[:,np.array([not i == 4 and  not i == 5  for i in range(1,23)])]
test = df_test.values 

print(train.shape)
print(train_ids.shape)
print(train_employed.shape)
print(train_data.shape)
print(test.shape)

(5212, 22)
(5212,)
(5212,)
(5212, 20)
(200, 20)


In [65]:
for i in range(0,len(train_employed)):
    print(train_employed[i],train_unemployed[i])

1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 0
1 0
1 0
1 0
0 1
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 1
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 1
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 1
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 0
1 0
0 1
1 0
1 0
1 0
1 0
0 1
1 0
1 0
1 0
0 0
1 0
1 0
1 0
1 0
1 0
0 1
1 0
1 0
1 0
1 0
1 0
1 0
0 0
1 0
1 0
1 0
1 0
1 0
0 1
0 0
1 0
1 0
1 0
0 0
1 0
1 0
1 0
1 0
1 0
0 1
1 0
0 0
1 0
1 0
1 0
1 0
1 0
0 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 0
1 0
1 0
0 1
1 0
1 0
1 0
1 0
1 0
0 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 0
1 0
0 1
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 1
1 0
1 0
1 0
1 0
1 0
1 0
0 1
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 1
0 1
1 0
1 0
1 0
0 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 1
0 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0


In [37]:
#Comparing of employed & unemployed for nonsensical data
ctr= 0
for i in range(0, len(train_employed)):
    if train_employed[i] == train_unemployed[i]:
        print(i, train_employed[i],train_unemployed[i])
        ctr+=1
print(ctr)
# Notes:
# Current understanding is that employed & unemployed are NOT exclusive

7 0 0
18 0 0
34 0 0
92 0 0
103 0 0
116 0 0
123 0 0
127 0 0
135 0 0
141 0 0
158 0 0
167 0 0
175 0 0
225 0 0
240 0 0
256 0 0
269 0 0
275 0 0
308 0 0
326 0 0
327 0 0
342 0 0
353 0 0
366 0 0
375 0 0
383 0 0
403 0 0
422 0 0
426 0 0
427 0 0
434 0 0
440 0 0
441 0 0
467 0 0
479 0 0
484 0 0
524 0 0
545 0 0
550 0 0
569 0 0
585 0 0
598 0 0
604 0 0
660 0 0
681 0 0
700 0 0
714 0 0
716 0 0
748 0 0
765 0 0
766 0 0
768 0 0
802 0 0
803 0 0
804 0 0
815 0 0
839 0 0
860 0 0
869 0 0
878 0 0
886 0 0
887 0 0
890 0 0
891 0 0
932 0 0
956 0 0
973 0 0
983 0 0
996 0 0
1055 0 0
1064 0 0
1107 0 0
1117 0 0
1118 0 0
1152 0 0
1206 0 0
1251 0 0
1263 0 0
1276 0 0
1313 0 0
1335 0 0
1341 0 0
1372 0 0
1373 0 0
1388 0 0
1397 0 0
1403 0 0
1428 0 0
1444 0 0
1457 0 0
1464 0 0
1489 0 0
1491 0 0
1517 0 0
1533 0 0
1557 0 0
1599 0 0
1604 0 0
1619 0 0
1643 0 0
1659 0 0
1664 0 0
1665 0 0
1675 0 0
1681 0 0
1683 0 0
1695 0 0
1739 0 0
1740 0 0
1762 0 0
1765 0 0
1831 0 0
1844 0 0
1867 0 0
1876 0 0
1897 0 0
1920 0 0
1928 0 0
1938 0 0
195

## Encoding & Train Test Split

In [68]:
#No encoding since gender, employment status, educ, states and such are already encoded

#Splitting
temp_train_ids, temp_test_ids, temp_train_data, temp_test_data, temp_train_employed, temp_test_employed = train_test_split(train_ids, train_data, train_employed, test_size=50, random_state=4)

## Preprocessing the data 

In [70]:
snv = sklearn.preprocessing.StandardScaler()
prep_temp_train_data = snv.fit_transform(temp_train_data)
prep_temp_test_data = snv.transform(temp_test_data)